In [1]:
import os
import json
import matplotlib.pyplot as plt
from datetime import datetime
import pandas as pd
import re
import random
import json
import os
import random
import re
import shutil
from datetime import datetime

import matplotlib.pyplot as plt
import pandas as pd
import requests
from reportlab.lib.enums import TA_LEFT
from reportlab.lib.pagesizes import letter
from reportlab.lib.styles import ParagraphStyle, getSampleStyleSheet
from reportlab.lib.units import inch
from reportlab.platypus import Paragraph, SimpleDocTemplate, Spacer

In [3]:
df = pd.read_csv("./data/incoming_first_year_application/incoming_first_year_application/incoming_first_year_application.csv")
print(len(df))
# df.head(5)df = pd.read_csv("./data/incoming_first_year_application/incoming_first_year_application/incoming_first_year_application.csv")
# print(len(df))
df.head(5)

10


,Serial number,Completed,Changed,Is draft,Submitted by: ID,Submitted by: Title,Submitted by: URL,Preferred Name,First Name,Middle Name,...,Experience with Web Design,General Programming Experience,1.,2.,3.,4.,5.,1..1,2..1,Upload Resume
0,940,2025-08-13 22:03:21,2025-08-13 22:03:21,0,0,NaN,NaN,Natalia,Minh Ngoc,NaN,...,3,4,Clough Hanson Gallery Attendant,"Business, Economics, and Politics and Law Depa...",Math & Computer Science,Human Resources,NaN,NaN,NaN,https://sites.rhodes.edu/system/files/webform/...
1,941,2025-08-16 18:54:28,2025-08-16 18:54:28,0,0,NaN,NaN,Grace,Grace,Olivia Ann,...,2,0,Library - Check-Out Desk,Enrollment Services,English Department,Residence Life,Student Engagement,NaN,NaN,https://sites.rhodes.edu/system/files/webform/...
2,942,2025-08-17 19:27:59,2025-08-17 19:27:59,0,0,NaN,NaN,Janethzie,Janethzie,NaN,...,0,0,Language center,Library,mailroom,gym,CityLynx,world relief,Mid south immigration advocates,https://sites.rhodes.edu/system/files/webform/...
3,943,2025-08-17 23:32:16,2025-08-17 23:32:16,0,0,NaN,NaN,Tabitha,Tabitha,Grace,...,2,3,Human Resources Staff,Religious Studies Staff,Music Department Staff,ANSO Department Staff,English Department Staff,NaN,N/A/,https://sites.rhodes.edu/system/files/webform/...
4,944,2025-08-18 01:36:33,2025-08-18 01:36:33,0,13436,easaf-29@rhodes.edu,https://sites.rhodes.edu/user/13436,Afia Maimuna,Afia,Maimuna,...,0,1,Career Services,Title IX / Campus Safety,Academic Affairs,Mailroom,Computer Science,NaN,NaN,https://sites.rhodes.edu/system/files/webform/...


In [13]:
print(len(df))
df['date'] = pd.to_datetime(df['Completed'])
df_sorted = df.sort_values(by=['Rhodes ID', 'date'], ascending=[True, False])
df = df_sorted.drop_duplicates(subset='Rhodes ID', keep='first')
len(df)

10


10

In [14]:
df.columns

Index(['Serial number', 'Completed', 'Changed', 'Is draft', 'Submitted by: ID',
       'Submitted by: Title', 'Submitted by: URL', 'Preferred Name',
       'First Name', 'Middle Name', 'Last Name', 'Preferred Pronouns',
       'Rhodes ID', 'Address 1', 'Address 2', 'City', 'State', 'Zip Code',
       'Cell Phone', 'Rhodes Email',
       'Can you provide proof, if hired, that you are eligible to work in the United States?',
       'Do you currently have a US social security number?  If no, we will help you apply for one after you arrive on campus.',
       'Are you available and interested in working in the evenings/on weekends?',
       'Are you willing to work outdoors?', 'Graduation Year from Rhodes',
       'Possible Majors (or N/A)', 'Computer Skills (hardware or software)',
       'Other Education/Training/Skills', 'Employer Name', 'Employer City',
       'Employer State', 'Employer Contact Phone', 'Supervisor Name',
       'Type of Business', 'Last Position Job Title', 'Employmen

In [15]:
cols = ["Rhodes ID", "Completed", "Last Name", "First Name", "Preferred Name", "Zip Code", "Zip Code", "Possible Majors (or N/A)", "Upload Resume", "Duties" + "Do you have experience in customer service? If so, please list. , Computer Skills (hardware or software) , Other Education/Training/Skills" + "1. 2. 3. 4. 5. 1..1 2..1"]

In [16]:
# data = input()
# existed_ids = set(data.split())
# len(existed_ids)
existed_ids = set()

In [17]:
def is_us_zip_code(zip_code):
    """
    Returns:
        bool: True if it matches a US zip code format, False otherwise.
    """
    # Regex for 5-digit US zip code
    # ^\d{5}$ matches exactly 5 digits from start to end
    zip_code = str(zip_code)
    five_digit_pattern = re.compile(r"^\d{5}$")

    # Regex for ZIP+4 US zip code
    # ^\d{5}-\d{4}$ matches 5 digits, a hyphen, and 4 digits from start to end
    zip_plus_four_pattern = re.compile(r"^\d{5}-\d{4}$")

    if five_digit_pattern.match(zip_code):
        return True
    elif zip_plus_four_pattern.match(zip_code):
        return True
    else:
        return False

def get_skills(row):
    skills = ["Duties", "Do you have experience in customer service? If so, please list.", "Computer Skills (hardware or software)", "Other Education/Training/Skills"]
    import math
    skills = [x for x in row[skills].tolist() if isinstance(x, str)]
    return " ".join(skills)

def get_pref(row):
    cols = "1. 2. 3. 4. 5. 1..1 2..1".split()
    data = [x for i, x in enumerate(row[cols].tolist()) if isinstance(x, str)]
    return "#".join(data)

In [18]:
df['Dom or Int'] = df.apply(lambda x : "Domestic" if is_us_zip_code(x["Zip Code"]) else "International", axis=1)
df['Skills'] = df.apply(lambda x : get_skills(x) , axis=1)
df["Preferences"] = df.apply(lambda x : get_pref(x) , axis=1)

In [19]:
cols = ["Rhodes Email", "Rhodes ID", "Completed", "Last Name", "First Name", "Preferred Name", "Dom or Int", "Zip Code", "Possible Majors (or N/A)", "Upload Resume", "Skills", "Preferences"]
filtered_df = df[~df["Rhodes ID"].isin(existed_ids)]
filtered_df = filtered_df[cols]
filtered_df.head(5)

,Rhodes Email,Rhodes ID,Completed,Last Name,First Name,Preferred Name,Dom or Int,Zip Code,Possible Majors (or N/A),Upload Resume,Skills,Preferences
3,meatg-26@rhodes.edu,05570682,2025-08-17 23:32:16,Meadors,Tabitha,Tabitha,Domestic,38112,Anthropology/ Sociology,https://sites.rhodes.edu/system/files/webform/...,"- Maintaining the email, front desk, and phone...",Human Resources Staff#Religious Studies Staff#...
8,lunaauroradiazlara@gmail.com,R060001674,2025-08-19 14:43:07,Diaz Lara,Luna,Luna,Domestic,85306,Physicology and Bussiness,https://sites.rhodes.edu/system/files/webform/...,Supervised and supported front-end cashiers to...,Residence Life #College Events #Athletics- BCL...
2,delja-27@rhodes.edu,R06000190,2025-08-17 19:27:59,Delgado,Janethzie,Janethzie,Domestic,38016,International Studies,https://sites.rhodes.edu/system/files/webform/...,- interact with customers\r\n- take orders \r\...,Language center #Library #mailroom #gym #CityL...
0,ngunm-28@rhodes.edu,R06001245,2025-08-13 22:03:21,Nguyen,Minh Ngoc,Natalia,Domestic,38112,Data Analytics and Economics,https://sites.rhodes.edu/system/files/webform/...,Scanned inbound and outbound mail and packages...,"Clough Hanson Gallery Attendant#Business, Econ..."
9,aureljue@berkeleyprep.org,R06001882,2025-08-19 23:24:08,Aurelien,Juelz,Juju,Domestic,33525,NaN,https://sites.rhodes.edu/system/files/webform/...,I have done community service with little kids...,Athletics- BCLC#Athletics- BCLC#Athletics- BCL...


In [20]:
print(len(existed_ids), len(filtered_df), len(df))
print(len(existed_ids), len(filtered_df["Rhodes ID"].unique()), len(df["Rhodes ID"].unique()))
# assert len(existed_ids) + len(filtered_df["Rhodes ID"].unique()) == len(df["Rhodes ID"].unique())

0 10 10
0 10 10


In [21]:
excel_file_name = "./data/New_App.xlsx"
if os.path.exists(excel_file_name):
    file_mode = 'a'
    writer_args = {'engine': 'openpyxl', 'mode': file_mode, 'if_sheet_exists': 'replace'}
else:
    file_mode = 'w'
    writer_args = {'engine': 'openpyxl', 'mode': file_mode}
    
with pd.ExcelWriter(excel_file_name, **writer_args) as writer:
    filtered_df.to_excel(writer, index=False)